In [1]:
!wget http://www.manythings.org/anki/fra-eng.zip


--2022-03-10 19:07:53--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.186.54, 104.21.92.44, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.186.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6532197 (6.2M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.23M  --.-KB/s    in 0.05s   

2022-03-10 19:07:53 (132 MB/s) - ‘fra-eng.zip’ saved [6532197/6532197]



In [2]:
import zipfile
zip = zipfile.ZipFile('fra-eng.zip')
zip.extractall()

In [3]:
import string, re
from unicodedata import normalize
from numpy import array, argmax
from pickle import load, dump
from numpy.random import rand, shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from nltk.translate.bleu_score import SmoothingFunction,corpus_bleu
smoothie = SmoothingFunction().method4

In [4]:
# load file into memory
def load_file(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [5]:
# split a loaded document into sentences
def splitting_sentence(doc):
	sentences = doc.strip().split('\n')
	pairs = [sentence.split('\t') for sentence in  sentences]
	return pairs

In [6]:
# cleaning a list of sentences and creating pairs

def clean_pairs(sentences):
	cleaned = list()
 
	# preparing regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))

	# preparing translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)

  # iterating over each pair
	for pair in sentences:
		clean_pair = list()
  
		for sentence in pair:
			# normalizing unicode characters
			sentence = normalize('NFD', sentence).encode('ascii', 'ignore')
			sentence = sentence.decode('UTF-8')
			# tokenizing on white space
			sentence = sentence.split()
			# converting to lowercase
			sentence = [word.lower() for word in sentence]
			# removing punctuation from each token
			sentence = [word.translate(table) for word in sentence]
			# removing non-printable chars form each token
			sentence = [re_print.sub('', w) for w in sentence]
			# removing tokens with numbers in them
			sentence = [word for word in sentence if word.isalpha()]
			# storing as string
			clean_pair.append(' '.join(sentence))
		cleaned.append(clean_pair)
	return array(cleaned)

In [7]:
def saving_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print(filename,': Saved')

In [8]:
# load dataset

filename = 'fra.txt'
doc = load_file(filename)

# split into english-french pairs
pairs = splitting_sentence(doc)

# clean sentences
clean_pairs = clean_pairs(pairs)

# save clean pairs to file
saving_clean_data(clean_pairs, 'english-french.pkl')

print('English','-->',"French")
# spot check
for i in range(25):
	print(clean_pairs[i,0],'-->',clean_pairs[i,1])

english-french.pkl : Saved
English --> French
go --> va
go --> marche
go --> bouge
hi --> salut
hi --> salut
run --> cours
run --> courez
run --> prenez vos jambes a vos cous
run --> file
run --> filez
run --> cours
run --> fuyez
run --> fuyons
run --> cours
run --> courez
run --> prenez vos jambes a vos cous
run --> file
run --> filez
run --> cours
run --> fuyez
run --> fuyons
who --> qui
wow --> ca alors
duck --> a terre
duck --> baissetoi


In [9]:
# load a clean dataset
def loading_cleaned_data(filename):
	return load(open(filename, 'rb'))

In [10]:
# load dataset
data = loading_cleaned_data('english-french.pkl')
print(data.shape) 

(192341, 3)


In [11]:
# reducing dataset size (scaling) 

new_data_size = 20000
dataset = data[:new_data_size, :]

# randomly shuffling the dataset to get proper training and testing data
shuffle(dataset)

# splitting into training and testing (90%-10%)
train, test = dataset[:18000], dataset[18000:]

# saving the cleaned data,train data and test data 
saving_clean_data(dataset, 'english-french-both.pkl')
saving_clean_data(train, 'english-french-train.pkl')
saving_clean_data(test, 'english-french-test.pkl')

english-french-both.pkl : Saved
english-french-train.pkl : Saved
english-french-test.pkl : Saved


In [12]:
# loading datasets and saving it into variables
dataset = loading_cleaned_data('english-french-both.pkl')
train = loading_cleaned_data('english-french-train.pkl')
test = loading_cleaned_data('english-french-test.pkl')

In [13]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [16]:
# preparing the english tokenizer

eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])

print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

# preparing the french tokenizer

fra_tokenizer = create_tokenizer(dataset[:, 1])
fra_vocab_size = len(fra_tokenizer.word_index) + 1
fra_length = max_length(dataset[:, 1])
print('French Vocabulary Size: %d' % fra_vocab_size)
print('French Max Length: %d' % (fra_length))

English Vocabulary Size: 3418
English Max Length: 5
French Vocabulary Size: 6977
French Max Length: 11


In [17]:
# Input and Output sequence must be encoded to integers and padded to the maximum phrase length
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	x = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	x = pad_sequences(x, maxlen=length, padding='post')
	return x

# One hot encoding to max phrase length
def one_hot_encoding(sequences, vocab_size):
	y_1 = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		y_1.append(encoded)
	y = array(y_1)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [18]:
# preparing training data
trainX = encode_sequences(fra_tokenizer, fra_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = one_hot_encoding(trainY, eng_vocab_size)

# prepare testing data
testX = encode_sequences(fra_tokenizer, fra_length, test[:, 1])
testY = encode_sequences(eng_tokenizer,eng_length, test[:, 0])
testY = one_hot_encoding(testY, eng_vocab_size)

In [19]:
print('training size:',trainX.shape,trainY.shape)
print('testing size:',testX.shape,testY.shape)

training size: (18000, 11) (18000, 5, 3418)
testing size: (2000, 11) (2000, 5, 3418)


In [20]:
def model_building(source_vocab, target_vocab, source_len, target_len, units):
	model = Sequential()
	model.add(Embedding(source_vocab, units, input_length=source_len, mask_zero=True))
	model.add(LSTM(units))
	model.add(RepeatVector(target_len))
	model.add(LSTM(units, return_sequences=True))
	model.add(TimeDistributed(Dense(target_vocab, activation='softmax')))
	return model

In [21]:
model = model_building(fra_vocab_size, eng_vocab_size, fra_length, eng_length, 512)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

In [22]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 512)           3572224   
                                                                 
 lstm (LSTM)                 (None, 512)               2099200   
                                                                 
 repeat_vector (RepeatVector  (None, 5, 512)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 5, 512)            2099200   
                                                                 
 time_distributed (TimeDistr  (None, 5, 3418)          1753434   
 ibuted)                                                         
                                                                 
Total params: 9,524,058
Trainable params: 9,524,058
Non-

In [23]:
# Stop model if accuracy of the model doesn't changes by more than 0.01 
# Patience = 5 : After each 5 epochs if no improvement is there then training will be stopped.
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_acc',patience= 5,min_delta=0.01)

In [24]:
# fit model
model.fit(trainX, trainY, epochs= 50, batch_size=25, validation_data=(testX, testY), verbose=2,callbacks=[es])

Epoch 1/50
720/720 - 17s - loss: 3.6005 - acc: 0.4640 - val_loss: 3.0783 - val_acc: 0.5159 - 17s/epoch - 24ms/step
Epoch 2/50
720/720 - 7s - loss: 2.7073 - acc: 0.5579 - val_loss: 2.5142 - val_acc: 0.5901 - 7s/epoch - 10ms/step
Epoch 3/50
720/720 - 7s - loss: 2.1475 - acc: 0.6158 - val_loss: 2.2130 - val_acc: 0.6217 - 7s/epoch - 10ms/step
Epoch 4/50
720/720 - 7s - loss: 1.7229 - acc: 0.6619 - val_loss: 2.0057 - val_acc: 0.6455 - 7s/epoch - 10ms/step
Epoch 5/50
720/720 - 7s - loss: 1.3543 - acc: 0.7086 - val_loss: 1.8474 - val_acc: 0.6713 - 7s/epoch - 10ms/step
Epoch 6/50
720/720 - 7s - loss: 1.0476 - acc: 0.7567 - val_loss: 1.7776 - val_acc: 0.6813 - 7s/epoch - 10ms/step
Epoch 7/50
720/720 - 8s - loss: 0.8008 - acc: 0.8031 - val_loss: 1.6799 - val_acc: 0.7066 - 8s/epoch - 10ms/step
Epoch 8/50
720/720 - 8s - loss: 0.6045 - acc: 0.8450 - val_loss: 1.6530 - val_acc: 0.7144 - 8s/epoch - 11ms/step
Epoch 9/50
720/720 - 8s - loss: 0.4621 - acc: 0.8757 - val_loss: 1.6546 - val_acc: 0.7188 - 8s

In [25]:
# mapping integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [26]:
# generating target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [27]:
# evaluating the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
  
  # Creating empty lists for actual phrases(French) and predicted phrases(English) 
  actual,predicted = list(),list()
  a,b,c = list(),list(),list()
  for i,source in enumerate(sources):

    # reshaping to the required size
    source = source.reshape((1, source.shape[0]))

    # predicting for the english tokenizer
    translation = predict_sequence(model, eng_tokenizer, source)
    # raw_dataset = raw_dataset[i].split(' ') 
    # print(raw_dataset[i][1])

    raw_src,raw_target = raw_dataset[i][1],raw_dataset[i][0]
    
    # First 10 Predictions
    if i <= 10:
      print('source = ',raw_src,'<--->', ' target = ',raw_target,'<--->','  predicted = ',translation)

    actual.append([raw_target.split()])
    predicted.append(translation.split())
  
  # calculating BLEU score
  print('-------------------------------------------')
  print('BLEU Score :')
  print('BLEU score-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie,auto_reweigh=False))

In [28]:
evaluate_model(model,eng_tokenizer,trainX,train)

source =  donnenous une chance <--->  target =  give us a chance <--->   predicted =  give us a chance
source =  laissezmoi faire ca <--->  target =  let me do that <--->   predicted =  let me do that
source =  notre point de vue est biaise <--->  target =  were biased <--->   predicted =  were biased
source =  personne ne peut le dire <--->  target =  no one can tell <--->   predicted =  no one can tell
source =  ils ne sont pas mal <--->  target =  theyre not bad <--->   predicted =  theyre not bad
source =  les chiens peuvent nager <--->  target =  dogs can swim <--->   predicted =  dogs can swim
source =  elle na pas tort <--->  target =  she is not wrong <--->   predicted =  she is not wrong
source =  jusqua quel point etesvous bon <--->  target =  how good are you <--->   predicted =  how good are you
source =  etesvous levee <--->  target =  are you up <--->   predicted =  are you up
source =  je vous envie <--->  target =  i envy you <--->   predicted =  i envy you
source =  to

In [29]:
evaluate_model(model, eng_tokenizer, testX, test)

source =  ne vous arretez pas <--->  target =  dont stop <--->   predicted =  keep moving
source =  on ma humiliee <--->  target =  i was humiliated <--->   predicted =  i was humiliated
source =  ditesle clairement <--->  target =  say it clearly <--->   predicted =  say it clearly
source =  sois tout simplement heureuse <--->  target =  just be happy <--->   predicted =  just be happy
source =  je ne suis pas content <--->  target =  i am not happy <--->   predicted =  im not happy
source =  je me sens prise de malaise <--->  target =  i feel faint <--->   predicted =  i feel faint
source =  il a raison ladessus <--->  target =  he has a point <--->   predicted =  he is away
source =  il ny avait pas de vent <--->  target =  it wasnt windy <--->   predicted =  he no blood
source =  je vends des chaussures <--->  target =  i sell shoes <--->   predicted =  i brought books
source =  tom detourna le regard <--->  target =  tom looked away <--->   predicted =  tom got away
source =  pass